# Performance summary over different `kissim` setups

Summarize performance of different `kissim` setups:

- Profiling vs. `kissim` AUCs: How well does `kissim` reflect profiling data?
- Phylogenetic `kissim` tree: How well do selected on- and off-targets cluster together?
- Top `kissim` ranks: How high do selected on- and off-targets rank in `kissim`?

DFG-in conformations only!

In [1]:
from pathlib import Path

import pandas as pd
import seaborn as sns
from Bio import Phylo

In [2]:
cm = sns.light_palette("blue", as_cmap=True)

In [3]:
HERE = Path(_dh[-1])  # noqa: F821
RESULTS = HERE / "../../results_archive"

## `kissim` setups

In [4]:
RUN_IDS = [
    "20210712",
    "20210804-1",
    "20210804-2",
    "20210804-3",
    "20210804-4",
    "20210804-5",
    "20210810-1",
    "20210810-2",
    "20210812-1",
    "20210812-2",
]
WEIGHTING_SCHEMES = ["15", "110", "101", "100"]
CLUSTERING_METHODS = ["ward", "average", "weighted"]

In [5]:
ON_OFF_PAIRS = [
    ["EGFR", ["SLK", "LOK", "GAK"]],  # Erlotinib
    ["SLK", ["LOK"]],  # Erlotinib
    ["DRAK2", ["CaMKK2"]],
    ["ABL2", ["AurA"]],  # VX-680/MK-0457
    ["ABL1", ["GAK"]],  # Dasatinib
    ["GAK", ["DAPK3"]],  # Inrebic
    ["AurC", ["KIT"]],  # Inlyta
    ["KIT", ["AMPKa2", "FMS"]],  # JNJ-28312141
    ["ABL1", ["BMPR1B"]]  # PD-173955
]

## Profiling vs. `kissim` AUCs

In [6]:
auc_dfs = {}
mean_df = []
median_df = []
std_df = []
for run_id in RUN_IDS:
    path = RESULTS / f"{run_id}/dfg_in"
    auc_df = pd.read_csv(path / "auc.csv")
    auc_dfs[run_id] = auc_df[["15", "100", "110", "101", "111"]]

    mean = auc_dfs[run_id].describe().loc["mean", :]
    mean.name = run_id
    mean_df.append(mean)

    median = auc_dfs[run_id].describe().loc["50%", :]
    median.name = run_id
    median_df.append(median)

    std = auc_dfs[run_id].describe().loc["std", :]
    std.name = run_id
    std_df.append(std)
mean_df = pd.concat(mean_df, axis=1)
median_df = pd.concat(median_df, axis=1)
std_df = pd.concat(std_df, axis=1)

### Mean

In [7]:
mean_df.style.background_gradient(cmap=cm, axis=None)

,20210712,20210804-1,20210804-2,20210804-3,20210804-4,20210804-5,20210810-1,20210810-2,20210812-1,20210812-2
15,0.729418,0.742915,0.741400,0.739638,0.728245,0.739012,0.732158,0.734549,0.730598,0.734957
100,0.737368,0.742814,0.740444,0.742814,0.737743,0.740444,0.734574,0.733362,0.733845,0.733818
110,0.732148,0.742441,0.742148,0.741624,0.735506,0.745352,0.733351,0.735241,0.731827,0.734931
101,0.716671,0.740182,0.739224,0.733893,0.692715,0.734812,0.728847,0.732000,0.727750,0.732394
111,0.707904,0.739671,0.737161,0.733173,0.691946,0.732063,0.729180,0.733298,0.727955,0.733283


In [8]:
mean_df.style.highlight_max(axis=None, color="yellow")

,20210712,20210804-1,20210804-2,20210804-3,20210804-4,20210804-5,20210810-1,20210810-2,20210812-1,20210812-2
15,0.729418,0.742915,0.741400,0.739638,0.728245,0.739012,0.732158,0.734549,0.730598,0.734957
100,0.737368,0.742814,0.740444,0.742814,0.737743,0.740444,0.734574,0.733362,0.733845,0.733818
110,0.732148,0.742441,0.742148,0.741624,0.735506,0.745352,0.733351,0.735241,0.731827,0.734931
101,0.716671,0.740182,0.739224,0.733893,0.692715,0.734812,0.728847,0.732000,0.727750,0.732394
111,0.707904,0.739671,0.737161,0.733173,0.691946,0.732063,0.729180,0.733298,0.727955,0.733283


In [9]:
mean_df.style.highlight_max(axis=0, color="yellow")

,20210712,20210804-1,20210804-2,20210804-3,20210804-4,20210804-5,20210810-1,20210810-2,20210812-1,20210812-2
15,0.729418,0.742915,0.741400,0.739638,0.728245,0.739012,0.732158,0.734549,0.730598,0.734957
100,0.737368,0.742814,0.740444,0.742814,0.737743,0.740444,0.734574,0.733362,0.733845,0.733818
110,0.732148,0.742441,0.742148,0.741624,0.735506,0.745352,0.733351,0.735241,0.731827,0.734931
101,0.716671,0.740182,0.739224,0.733893,0.692715,0.734812,0.728847,0.732000,0.727750,0.732394
111,0.707904,0.739671,0.737161,0.733173,0.691946,0.732063,0.729180,0.733298,0.727955,0.733283


In [10]:
mean_df.style.highlight_max(axis=1, color="yellow")

,20210712,20210804-1,20210804-2,20210804-3,20210804-4,20210804-5,20210810-1,20210810-2,20210812-1,20210812-2
15,0.729418,0.742915,0.741400,0.739638,0.728245,0.739012,0.732158,0.734549,0.730598,0.734957
100,0.737368,0.742814,0.740444,0.742814,0.737743,0.740444,0.734574,0.733362,0.733845,0.733818
110,0.732148,0.742441,0.742148,0.741624,0.735506,0.745352,0.733351,0.735241,0.731827,0.734931
101,0.716671,0.740182,0.739224,0.733893,0.692715,0.734812,0.728847,0.732000,0.727750,0.732394
111,0.707904,0.739671,0.737161,0.733173,0.691946,0.732063,0.729180,0.733298,0.727955,0.733283


### Median

In [11]:
median_df.style.background_gradient(cmap=cm, axis=None)

,20210712,20210804-1,20210804-2,20210804-3,20210804-4,20210804-5,20210810-1,20210810-2,20210812-1,20210812-2
15,0.769479,0.787637,0.779823,0.787594,0.764674,0.784618,0.775239,0.776538,0.778683,0.780000
100,0.786876,0.795961,0.784854,0.795961,0.779133,0.784854,0.779885,0.776538,0.782162,0.780000
110,0.766542,0.793199,0.790160,0.790582,0.777778,0.784037,0.769600,0.776538,0.774922,0.780000
101,0.753931,0.789474,0.781529,0.776316,0.701081,0.778246,0.760425,0.775637,0.759579,0.780000
111,0.726319,0.784902,0.774071,0.776316,0.701876,0.775046,0.762348,0.776748,0.759601,0.780000


In [12]:
median_df.style.highlight_max(axis=None, color="yellow")

,20210712,20210804-1,20210804-2,20210804-3,20210804-4,20210804-5,20210810-1,20210810-2,20210812-1,20210812-2
15,0.769479,0.787637,0.779823,0.787594,0.764674,0.784618,0.775239,0.776538,0.778683,0.780000
100,0.786876,0.795961,0.784854,0.795961,0.779133,0.784854,0.779885,0.776538,0.782162,0.780000
110,0.766542,0.793199,0.790160,0.790582,0.777778,0.784037,0.769600,0.776538,0.774922,0.780000
101,0.753931,0.789474,0.781529,0.776316,0.701081,0.778246,0.760425,0.775637,0.759579,0.780000
111,0.726319,0.784902,0.774071,0.776316,0.701876,0.775046,0.762348,0.776748,0.759601,0.780000


In [13]:
median_df.style.highlight_max(axis=1, color="yellow")

,20210712,20210804-1,20210804-2,20210804-3,20210804-4,20210804-5,20210810-1,20210810-2,20210812-1,20210812-2
15,0.769479,0.787637,0.779823,0.787594,0.764674,0.784618,0.775239,0.776538,0.778683,0.780000
100,0.786876,0.795961,0.784854,0.795961,0.779133,0.784854,0.779885,0.776538,0.782162,0.780000
110,0.766542,0.793199,0.790160,0.790582,0.777778,0.784037,0.769600,0.776538,0.774922,0.780000
101,0.753931,0.789474,0.781529,0.776316,0.701081,0.778246,0.760425,0.775637,0.759579,0.780000
111,0.726319,0.784902,0.774071,0.776316,0.701876,0.775046,0.762348,0.776748,0.759601,0.780000


### Standard deviation

In [14]:
std_df.style.background_gradient(cmap=cm, axis=None)

,20210712,20210804-1,20210804-2,20210804-3,20210804-4,20210804-5,20210810-1,20210810-2,20210812-1,20210812-2
15,0.146028,0.146782,0.147242,0.144187,0.141115,0.145573,0.140730,0.140837,0.142213,0.141023
100,0.154774,0.151072,0.150506,0.151072,0.150279,0.150506,0.145065,0.141067,0.145576,0.141190
110,0.148565,0.149837,0.148428,0.148939,0.145607,0.148299,0.143323,0.140535,0.144172,0.140227
101,0.140226,0.144862,0.144850,0.142372,0.127371,0.143996,0.136722,0.140287,0.138263,0.140921
111,0.138169,0.143709,0.143580,0.141556,0.126978,0.143610,0.136699,0.139429,0.137759,0.140164


In [15]:
std_df.style.highlight_max(axis=None, color="yellow")

,20210712,20210804-1,20210804-2,20210804-3,20210804-4,20210804-5,20210810-1,20210810-2,20210812-1,20210812-2
15,0.146028,0.146782,0.147242,0.144187,0.141115,0.145573,0.140730,0.140837,0.142213,0.141023
100,0.154774,0.151072,0.150506,0.151072,0.150279,0.150506,0.145065,0.141067,0.145576,0.141190
110,0.148565,0.149837,0.148428,0.148939,0.145607,0.148299,0.143323,0.140535,0.144172,0.140227
101,0.140226,0.144862,0.144850,0.142372,0.127371,0.143996,0.136722,0.140287,0.138263,0.140921
111,0.138169,0.143709,0.143580,0.141556,0.126978,0.143610,0.136699,0.139429,0.137759,0.140164


In [16]:
std_df.style.highlight_max(axis=0, color="yellow")

,20210712,20210804-1,20210804-2,20210804-3,20210804-4,20210804-5,20210810-1,20210810-2,20210812-1,20210812-2
15,0.146028,0.146782,0.147242,0.144187,0.141115,0.145573,0.140730,0.140837,0.142213,0.141023
100,0.154774,0.151072,0.150506,0.151072,0.150279,0.150506,0.145065,0.141067,0.145576,0.141190
110,0.148565,0.149837,0.148428,0.148939,0.145607,0.148299,0.143323,0.140535,0.144172,0.140227
101,0.140226,0.144862,0.144850,0.142372,0.127371,0.143996,0.136722,0.140287,0.138263,0.140921
111,0.138169,0.143709,0.143580,0.141556,0.126978,0.143610,0.136699,0.139429,0.137759,0.140164


In [17]:
std_df.style.highlight_max(axis=1, color="yellow")

,20210712,20210804-1,20210804-2,20210804-3,20210804-4,20210804-5,20210810-1,20210810-2,20210812-1,20210812-2
15,0.146028,0.146782,0.147242,0.144187,0.141115,0.145573,0.140730,0.140837,0.142213,0.141023
100,0.154774,0.151072,0.150506,0.151072,0.150279,0.150506,0.145065,0.141067,0.145576,0.141190
110,0.148565,0.149837,0.148428,0.148939,0.145607,0.148299,0.143323,0.140535,0.144172,0.140227
101,0.140226,0.144862,0.144850,0.142372,0.127371,0.143996,0.136722,0.140287,0.138263,0.140921
111,0.138169,0.143709,0.143580,0.141556,0.126978,0.143610,0.136699,0.139429,0.137759,0.140164


## Top `kissim` ranks

In [18]:
def get_ranks(distance_matrix, rank_from, rank_to):
    """
    Get ranks for all kinases w.r.t. to a query kinase.
    """

    ranks = pd.concat(
        [
            distance_matrix[rank_from].sort_values(),
            distance_matrix[rank_from].sort_values().rank(),
        ],
        axis=1,
    )
    ranks.columns = ["distance", "rank"]
    if rank_to is not None:
        ranks = ranks.loc[rank_to, :]
    pair_names = [f"{rank_from}-{rank_to}" for i in rank_to]
    return ranks, pair_names

In [48]:
%%time

results_list = []

for run_id in RUN_IDS[1:]:
    for weighting in WEIGHTING_SCHEMES:
        results = []
        columns = []
        results.extend([run_id, weighting])

        kinase_matrix_path = RESULTS / f"{run_id}/dfg_in/fingerprint_distances_to_kinase_matrix.csv"
        kinase_matrix = pd.read_csv(kinase_matrix_path, index_col=0)

        for pair in ON_OFF_PAIRS:
            ranks, pair_names = get_ranks(kinase_matrix, pair[0], pair[1])
            results.extend(ranks["rank"].to_list())

        results_list.append(results)

CPU times: user 1.05 s, sys: 28 ms, total: 1.07 s
Wall time: 1.07 s


In [49]:
matrix_ranks_df = pd.DataFrame(
    results_list,
    columns=["run_id", "weighting"] + [f"{i[0]}-{j}" for i in ON_OFF_PAIRS for j in i[1]]
)
matrix_ranks_df = matrix_ranks_df.set_index(["run_id", "weighting"])
cm = sns.light_palette("blue", as_cmap=True, reverse=True)

In [50]:
matrix_ranks_df.style.applymap(lambda x: "background-color : yellow" if x < 25 else "")

In [51]:
matrix_ranks_df.style.background_gradient(cmap=cm, axis=None)

## Phylogenetic `kissim` tree

In [23]:
import itertools
import numpy as np

In [24]:
def pairs_to_symmetric_matrix(pairs):
    """
    Create symmetric matrix with diagonal of 0.0 from pair combinations.
    """
    
    pairs1 = pd.DataFrame(pairs)
    pairs2 = pairs1[[1, 0, 2]]
    pairs2.columns = [0, 1, 2]
    pairs_all = pd.concat([pairs1, pairs2])
    matrix = pairs_all.pivot(index=0, columns=1, values=2)
    np.fill_diagonal(matrix.values, 0.0)
    symmetic_matrix = pd.DataFrame(matrix.values, columns=matrix.columns, index=matrix.index)
    return symmetic_matrix

In [25]:
def get_tree_distance_matrix(tree):
    """
    Get a matrix of all-against-all kinase distances in the kinase tree.
    """

    kinases = [clade.name for clade in tree.get_terminals()]
    kinase_pairs = itertools.combinations(kinases, 2)

    kinase_pairs_tree_distances = []

    for kinase1, kinase2 in kinase_pairs:
        kinase_pairs_tree_distances.append([kinase1, kinase2, tree.distance(kinase1, kinase2)])

    tree_distance_matrix = pairs_to_symmetric_matrix(kinase_pairs_tree_distances)

    return tree_distance_matrix

In [26]:
%%time

results_list = []

for run_id in RUN_IDS:
    print(run_id)
    for weighting in WEIGHTING_SCHEMES:
        for cmethod in CLUSTERING_METHODS:
            results = []
            results.extend([run_id, weighting, cmethod])

            tree_path = RESULTS / f"{run_id}/dfg_in/trees/tree_0.8_{weighting}_{cmethod}.tree"
            kissim_tree = Phylo.read(tree_path, "newick")
            tree_distance_matrix = get_tree_distance_matrix(kissim_tree)
            
            for pair in ON_OFF_PAIRS:
                ranks, pair_names = get_ranks(tree_distance_matrix, pair[0], pair[1])
                results.extend(ranks["rank"].to_list())

            results_list.append(results)

20210712
20210804-1
20210804-2
20210804-3
20210804-4
20210804-5
20210810-1
20210810-2
20210812-1
20210812-2
CPU times: user 22min 43s, sys: 277 ms, total: 22min 43s
Wall time: 22min 47s


In [28]:
tree_ranks_df = pd.DataFrame(
    results_list,
    columns=["run_id", "weighting", "cmethod"] + [f"{i[0]}-{j}" for i in ON_OFF_PAIRS for j in i[1]]
)
tree_ranks_df = tree_ranks_df.set_index(["run_id", "weighting", "cmethod"])
cm = sns.light_palette("blue", as_cmap=True, reverse=True)

In [29]:
tree_ranks_df.style.applymap(lambda x: "background-color : yellow" if x < 25 else "")

In [40]:
tree_ranks_df.loc[:, "15", :].style.background_gradient(cmap=cm, axis=0)